In [2]:
from langchain.llms import HuggingFaceHub
import os
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [6]:
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",  # Or deepseek
    huggingfacehub_api_token=token,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

/tmp/ipykernel_9163/3665916698.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [116]:
text = """ Harish is a very good athlete and has a stamina of 10.5 with a weight of 75kgs"""


In [118]:
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import ResponseSchema

In [120]:
stamina_schema = ResponseSchema(
    name="stamina",
    description = "stamina of the person given in the text in integer format"
)
weight_schema = ResponseSchema(
    name="weight",
    description = "weight of the person given in the text in integer format"
)

response_schemas = [stamina_schema,weight_schema]

In [122]:
print(response_schemas)

[ResponseSchema(name='stamina', description='stamina of the person given in the text in integer format', type='string'), ResponseSchema(name='weight', description='weight of the person given in the text in integer format', type='string')]


In [124]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"stamina": string  // stamina of the person given in the text in integer format
	"weight": string  // weight of the person given in the text in integer format
}
```


In [126]:
import re
format_instructions = re.sub(r'//.*', '', format_instructions)

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"stamina": string  
	"weight": string  
}
```


In [128]:
template_string = """Extract the required data from the text below (between triple backticks).\ 
Return output in a markdown code block with JSON format.\
The "stamina" and "weight" must be strings representing integers.\

            text : ```{text}```\
            format_instructions:{format_instructions}"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_9163/926391090.py:1: SyntaxWarning: invalid escape sequence '\ '
  template_string = """Extract the required data from the text below (between triple backticks).\


In [130]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)


In [132]:

request_customer = prompt_template.format_messages(text=text,format_instructions=format_instructions)
print(request_customer[0].content)

Extract the required data from the text below (between triple backticks).\ 
Return output in a markdown code block with JSON format.The "stamina" and "weight" must be strings representing integers.
            text : ``` Harish is a very good athlete and has a stamina of 10.5 with a weight of 75kgs```            format_instructions:The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"stamina": string  
	"weight": string  
}
```


In [134]:
reply = llm(request_customer[0].content)
print(reply)

Extract the required data from the text below (between triple backticks).\ 
Return output in a markdown code block with JSON format.The "stamina" and "weight" must be strings representing integers.
            text : ``` Harish is a very good athlete and has a stamina of 10.5 with a weight of 75kgs```            format_instructions:The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"stamina": string  
	"weight": string  
}
```
The "stamina" and "weight" keys should have integer values converted to string.

```json
{
	"stamina": "10.5",
	"weight": "75"
}
```


In [135]:
type(reply)

str

In [136]:
parsed_output = output_parser.parse(reply)  # This is what turns str ➝ dict
print(parsed_output)

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 